In [1]:
# Set directories 
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)
data_dir = os.path.join(os.getcwd(), '..', 'data')
#from IPython.core.debugger import set_trace

# Import Functions
import pandas as pd
from d00_utils.example_func import greet


With the following code I want to prepare the training set so that I can fit a first simple model. First I have to correctly structure the training set





Prior concat train
Left join orders 
orders_and_items = User, order_id,item, order_nr, more info 

train_order_user = train, group by user, max order  
max_order_item = Group by user Id, product get max order 
Train order user left join max order item if max order == train order 
Train and prior order group by user item max (order)

In [103]:
order_product_prior = pd.read_csv(os.path.join(data_dir,"01_raw/order_products__prior.csv"))
order_product_train = pd.read_csv(os.path.join(data_dir,"01_raw/order_products__train.csv"))
orders = pd.read_csv(os.path.join(data_dir,"01_raw/orders.csv"))
products = pd.read_csv(os.path.join(data_dir,"01_raw/products.csv"))
aisles = pd.read_csv(os.path.join(data_dir,"01_raw/aisles.csv"))
departments = pd.read_csv(os.path.join(data_dir,"01_raw/departments.csv"))


In [104]:
# union all all info about which product in which order 
order_product_train['eval_set'] = 'train'
order_product_prior['eval_set'] = 'prior'
order_product_gesamt = pd.concat([order_product_prior,order_product_train])

# reduce order to orders of only a small subset of users to test 
orders = orders.loc[orders.user_id <= 100,:]

# reduce order_product_gesamt to only include these few users (inner join)
# add additional order information to order_product_gesamt 
order_product_gesamt = order_product_gesamt.merge(orders.loc[:, ['order_id','user_id','order_number'] ],how="inner",on = "order_id")


* order_product_train bzw. order_product_prior contain the products (product_Id) which were in a specific order
* orders contains more specific information about each order
    * which user
    * which set the order is in (train, test, prior) 
    * information about the timing 


get all orders of customers


In [ ]:
order_product_gesamt.head(2)

In [ ]:
orders.head(2)

Create Trainset

* For each item a user purchased get the last order it was included
* 


In [65]:
def prepare_flatfile(order_product_gesamt,orders,which_set):
    # get all items previously ordered items - in train only check items which were ordered prior 
    # (automatically excludes products which were ordered in train for the first time)
    if which_set == 'train':
        order_product_gesamt = order_product_gesamt.loc[order_product_gesamt.eval_set != 'train']
    flatfile = order_product_gesamt.loc[:,['user_id','product_id']].drop_duplicates().reset_index()
    
    # get all orders from the specific set - some users are only in train or test (or neither?) 
    rel_orders = orders.loc[orders.eval_set == which_set,['user_id','order_id','order_number']].rename(
       columns={'order_number': 'ff_order_number', 'order_id': 'ff_order_id'})
    
    flatfile = flatfile.merge(rel_orders,how = 'inner',on = 'user_id')
        
    return(flatfile)

In [64]:
def prepare_target(train,order_product_gesamt):
    # get all orders of user
        
    prod_in_train = order_product_gesamt.loc[order_product_gesamt.eval_set == 'train',['order_id','product_id','eval_set']]
        
    train = train.merge(prod_in_train,left_on = ['ff_order_id','product_id'],right_on = ['order_id','product_id'] ,how='left')
        
    train['target'] = train['eval_set'].apply(lambda x: 0 if pd.isnull(x) else 1)
    
    train = train.drop(columns = ['order_id','eval_set'])
        
    return(train)

In [66]:
train = prepare_flatfile(order_product_gesamt,orders,'train')
train = prepare_target(train,order_product_gesamt)


In [69]:
train.loc[(train.user_id) == 90 & (train.target == 1)]
order_product_gesamt.loc[(order_product_gesamt.user_id == 90) & (order_product_gesamt.eval_set == 'train')]

,order_id,product_id,add_to_cart_order,reordered,eval_set,user_id,order_number
13215,1318796,33318,1,1,train,90,72
13216,1318796,32648,2,1,train,90,72
13217,1318796,36994,3,1,train,90,72
13218,1318796,19204,4,1,train,90,72
13219,1318796,939,5,1,train,90,72


In [75]:
def get_item_frequency(train,order_product_gesamt,which_set): 
    
    if which_set == 'train':
        order_product_gesamt = order_product_gesamt.loc[order_product_gesamt.eval_set != 'train']
    
    orders_per_item = order_product_gesamt.groupby(
        ['user_id','product_id']).size().reset_index(name='user_item_order_freq')
        
    train = train.merge(orders_per_item,how = 'left',on = ['user_id','product_id'])
    return(train)

In [77]:
train = get_item_frequency(train,order_product_gesamt,'train')

In [84]:
def get_last_order_of_item(order_product_gesamt,which_set):
    # df is a dataframe of all product orders excluding the test orders
    # for each user and all of his ordered items: what was the last order an item was ordered in
    if which_set == 'train':
        order_product_gesamt = order_product_gesamt.loc[order_product_gesamt.eval_set != 'train']
    
    last_order_item = order_product_gesamt.groupby(['user_id','product_id']).agg( {'order_number': 'max'} ).reset_index()
    # get the order ids for each order of each user 
    id_of_each_order = order_product_gesamt.loc[:,['user_id','order_number','order_id']].drop_duplicates().reset_index(drop=True)
    last_order_item = last_order_item.merge(id_of_each_order,how="inner",on = ['user_id','order_number'])
    last_order_item = last_order_item.rename(columns={'order_number': 'last_order_number', 'order_id': 'last_order_id'})
    return(last_order_item)

In [93]:
orders_pre_train = get_last_order_of_item(order_product_gesamt,'train')
train = train.merge(orders_pre_train,how = 'left', on = ['user_id','product_id'])

In [129]:
def get_time_to_last_order(train,orders,which_set):
    # for every user,item get the last order before the train order
    # for each order calculate the distance in days to the train order 
    orders = orders.loc[orders.eval_set != "test"]
    
    orders = orders.set_index('order_number').sort_index(ascending = False)

    orders['order_days_cumsum'] = orders.groupby(
    'user_id')['days_since_prior_order'].transform(pd.Series.cumsum)
    orders['order_days_cumsum'] = orders.groupby('user_id')['order_days_cumsum'].shift(periods=1).fillna(value=0)
    orders = orders.reset_index()

    orders = orders.loc[:,['order_id','order_days_cumsum']]
    
    train = train.merge(orders,how='left',left_on = 'last_order_id',right_on = 'order_id')
    
    return(train)

In [131]:
train = get_time_to_last_order(train,orders,'train')